<a href="https://colab.research.google.com/github/chorltonm/fa-cup-upsets/blob/main/notebooks/source_apis/football_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Libaries

import requests
import math
import json
import os
import pandas as pd

from google.cloud import bigquery
from google.oauth2 import service_account
from google.colab import drive
from google.colab import userdata

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

# Change default ouput directory
os.chdir('/content/drive/MyDrive/birkbeck_msc-project/output_files')

Mounted at /content/drive


In [3]:
# SportDevs Bearer Token API Authentication
userdata.get('sportdevs_bearer')
bearer_token = userdata.get('sportdevs_bearer')

headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {bearer_token}"
}


# Google Service Account

# Load the JSON key from local Google Collab file
key = json.load(open('/content/drive/MyDrive/service_account.json', 'r'))

# Authenticate using the loaded key
credentials = service_account.Credentials.from_service_account_info(key)

# Set up the BigQuery client with the credentials to project
client = bigquery.Client(credentials=credentials, project='birkbeck-msc-project-422917')

In [4]:
# Set filter variables
country = 'England'
league_name = 'Premier League'

# LEAGUE  DATA
# Get League ID
leagues_endpoint = f"https://football.sportdevs.com/leagues?class_name=eq.{country}&name=eq.{league_name}"
leagues_response = requests.get(leagues_endpoint, headers=headers)
leagues_data = leagues_response.json()

print(leagues_data)

# Get Seasons Data for League One and save seasons data to extract file
from_date = '2008-07-01'

seasons_list = []

 # SEASONS DATA
with open(f"{league_name}_seasons_data.json", 'w') as seasons_file:
  for league in leagues_data:
      league_id = league['id']

      # Get seasons for the league
      seasons_url = f"https://football.sportdevs.com/seasons?league_id=eq.{league_id}&start_time=gte.{from_date}"
      seasons_response = requests.get(seasons_url, headers=headers)
      seasons_data = seasons_response.json()

      matches_list = []
      lineups_list = []
      season_standings_list = []

      for season in seasons_data:
          seasons_file.write(f"{json.dumps(season)}\n")
          seasons_list.append(season)
          season_id = season['id']

          # LEAGUE STANDINDS DATA
          # Only run the season_standings section if the league is not FA Cup
          if league_name != 'FA Cup':
            season_standings_url = f"https://football.sportdevs.com/standings?type=eq.total&season_id=eq.{season_id}"
            season_standings_response = requests.get(season_standings_url, headers=headers)
            season_standings_data = season_standings_response.json()

            for season in season_standings_data:
              season_standings_list.append(season)
              print(*season_standings_list, sep="\n")

          # MATCH DATA
          # Get all matches for season ID and maximum 50 results per request as per api documentation
            limit = 50
            offset = 0
            number_of_matches = 0

          while True:
            matches_url = f"https://football.sportdevs.com/matches?season_id=eq.{season_id}&limit={limit}&offset={offset}"
            matches_response = requests.get(matches_url, headers=headers)
            matches_data = matches_response.json()

            for matches in matches_data:
                matches_list.append(matches)

                if 'lineups_id' in matches:
                    lineups_list.append(matches['lineups_id'])

            offset += limit
            number_of_matches += len(matches_data)
            print(number_of_matches)

            if len(matches_data) < limit:
                break

# Write all season teams from list to file at once
with open(f"{league_name}_season_standings_data.json", 'w') as season_standings_file:
  for season in season_standings_list:
      season_standings_file.write(f"{json.dumps(season)}\n")

# Write all matches from list to file at once
with open(f"{league_name}_matches_data.json", 'w') as matches_file:
  for match in matches_list:
      matches_file.write(f"{json.dumps(match)}\n")

lineup_ids = sorted(lineups_list)
total_seasons = len(seasons_list)
total_matches = len(matches_list)
total_lineup_ids = len(lineup_ids)
print(f"Total seasons: {total_seasons}")
print(f"Total mactches: {total_matches}")
print(f"Total line up ids: {total_lineup_ids}")

#print(*matches_list, sep="\n")


[{'id': 16, 'name': 'Premier League', 'importance': 1, 'level': 1, 'current_champion_team_id': 12, 'current_champion_team_name': 'Manchester City', 'current_champion_team_hash_image': '302cb62c447e870778c786eebf83f5fee881e1e289f2bb552fb2900acb4b5a35', 'current_champion_team_num_titles': 10, 'teams_most_titles': [{'team_id': 14, 'team_name': 'Manchester United', 'team_hash_image': '93af4a34b678aeb2dfded46e78c4b89de583eebc7085072be37e7a1c64b790cd'}], 'most_titles': 20, 'primary_color': '#3c1c5a', 'secondary_color': '#f80158', 'lower_leagues': [{'league_id': 172, 'league_name': 'Championship', 'league_hash_image': 'f6e1e146682aca6bc217db36a4e0a4cff4a74e5df43a43353140c3fe647bf730'}], 'start_league': '2024-08-16T00:00:00+00:00', 'end_league': '2025-05-25T00:00:00+00:00', 'hash_image': '69a3f23712e51ed08e223d7d438c358acea5b8eac0b75cb1526589024fcd255b', 'class_id': 158, 'class_name': 'England', 'class_hash_image': '39bcba0feb303912d90fe5917cdfbabab8d0ac780d3c73f387344293b2e3ae58'}]
{'id': 14,

In [5]:
# Load seasons file to BIQ QUERY extract_layer seasons table

# Set the dataset and table name
dataset_name = 'extract_layer'
table_name = 'api_sportdevs_fb_seasons'
table_ref = client.dataset(dataset_name).table(table_name)

# Load the final data file into the BiqQuery table
job_config = bigquery.LoadJobConfig(
  source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
  write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
  autodetect=True
)

with open(f"{league_name}_seasons_data.json", 'rb') as lineups_file:
              job = client.load_table_from_file(
                    lineups_file, table_ref, job_config=job_config
                )
# Wait for the job to complete
job.result()
total_rows = job.output_rows
print(f"Big Query Rows {total_rows} processed into {dataset_name}.{table_name}")

Big Query Rows 17 processed into extract_layer.api_sportdevs_fb_seasons


In [6]:
# Load seasons data into BIQ QUERY load_layer seasons table

# Set the target load dataset and table names
load_dataset_name = 'extract_layer'
load_table_name = 'all_api_sportdevs_fb_seasons'
table_ref = client.dataset(load_dataset_name).table(load_table_name)

# Delete rows already inserted
delete_query = f"DELETE FROM `{load_dataset_name}.{load_table_name}` WHERE id IN (SELECT DISTINCT id FROM `{dataset_name}.{table_name}`)"
delete_job = client.query(delete_query)
delete_result = delete_job.result()
total_rows_deleted = delete_result.num_dml_affected_rows

if total_rows_deleted == 0:
    print(f"No rows were deleted from {load_dataset_name}.{load_table_name}.")
else:
   print(f"Total rows deleted from {load_dataset_name}.{load_table_name}: {total_rows_deleted}")

# Define the load job configuration
job_config = bigquery.QueryJobConfig(
    destination=table_ref,
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
    schema_update_options=[
        bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION
    ]
)

# Define the SQL query
insert_query = f"SELECT * FROM `{dataset_name}.{table_name}`"

# Run the load job and wait for the job to complete
load_job = client.query(insert_query, job_config=job_config)
job.result()
total_rows = job.output_rows
print(f"Big Query Rows {total_rows} processed into {load_dataset_name}.{load_table_name}")

Total rows deleted from extract_layer.all_api_sportdevs_fb_seasons: 17
Big Query Rows 17 processed into extract_layer.all_api_sportdevs_fb_seasons


In [7]:
# Load standings teams file to BIQ QUERY extract_layer season teams table

if league_name != 'FA Cup':
  # Set the dataset and table name
  dataset_name = 'extract_layer'
  table_name = 'api_sportdevs_fb_standings'
  table_ref = client.dataset(dataset_name).table(table_name)

  # Load the final data file into the BiqQuery table
  job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    autodetect=True
  )

  with open(f"{league_name}_season_standings_data.json", 'rb') as lineups_file:
                job = client.load_table_from_file(
                      lineups_file, table_ref, job_config=job_config
                  )
  # Wait for the job to complete
  job.result()
  total_rows = job.output_rows
  print(f"Big Query Rows {total_rows} processed into {dataset_name}.{table_name}")

Big Query Rows 17 processed into extract_layer.api_sportdevs_fb_standings


In [8]:
# Load standings data into BIQ QUERY load_layer season teams table

if league_name != 'FA Cup':
  # Set the target load dataset and table names
  load_dataset_name = 'extract_layer'
  load_table_name = 'all_api_sportdevs_fb_standings'
  table_ref = client.dataset(load_dataset_name).table(load_table_name)

  # Delete rows already inserted
  delete_query = f"DELETE FROM `{load_dataset_name}.{load_table_name}` WHERE id IN (SELECT DISTINCT id FROM `{dataset_name}.{table_name}`)"
  delete_job = client.query(delete_query)
  delete_result = delete_job.result()
  total_rows_deleted = delete_result.num_dml_affected_rows

  if total_rows_deleted == 0:
      print(f"No rows were deleted from {load_dataset_name}.{load_table_name}.")
  else:
    print(f"Total rows deleted from {load_dataset_name}.{load_table_name}: {total_rows_deleted}")

  # Define the load job configuration
  job_config = bigquery.QueryJobConfig(
      destination=table_ref,
      write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
      schema_update_options=[
          bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION
      ]
  )

  # Define the SQL query
  insert_query = f"SELECT * FROM `{dataset_name}.{table_name}`"

  # Run the load job and wait for the job to complete
  load_job = client.query(insert_query, job_config=job_config)
  job.result()
  total_rows = job.output_rows
  print(f"Big Query Rows {total_rows} processed into {load_dataset_name}.{load_table_name}")

Total rows deleted from extract_layer.all_api_sportdevs_fb_standings: 17
Big Query Rows 17 processed into extract_layer.all_api_sportdevs_fb_standings


In [9]:
# Load matches file to BIQ QUERY extract_layer matches table

# Set the dataset and table names
dataset_name = 'extract_layer'
table_name = 'api_sportdevs_fb_matches'
table_ref = client.dataset(dataset_name).table(table_name)

#table = client.get_table(table_ref)

# Truncate the table to remove all existing rows
client.query(f"TRUNCATE TABLE `{dataset_name}.{table_name}`").result()


# Load the final data file into the BiqQuery table
job_config = bigquery.LoadJobConfig(
  source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
  write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
  autodetect=True
)

with open(f"{league_name}_matches_data.json", 'rb') as lineups_file:
              job = client.load_table_from_file(
                    lineups_file, table_ref, job_config=job_config
                )
# Wait for the job to complete
job.result()
total_rows = job.output_rows
print(f"Big Query Rows {total_rows} processed into {dataset_name}.{table_name}")

Big Query Rows 6634 processed into extract_layer.api_sportdevs_fb_matches


In [10]:
# Load matches extract data into BIQ QUERY load_layer matches table

# Set the table reference
load_dataset_name = 'extract_layer'
load_table_name = 'all_api_sportdevs_fb_matches'
table_ref = client.dataset(load_dataset_name).table(load_table_name)

# Delete rows already inserted
delete_query = f"DELETE FROM `{load_dataset_name}.{load_table_name}` WHERE id IN (SELECT DISTINCT id FROM `{dataset_name}.{table_name}`)"
delete_job = client.query(delete_query)
delete_result = delete_job.result()
total_rows_deleted = delete_result.num_dml_affected_rows

if total_rows_deleted == 0:
    print(f"No rows were deleted from {load_dataset_name}.{load_table_name}.")
else:
   print(f"Total rows deleted from {load_dataset_name}.{load_table_name}: {total_rows_deleted}")

# Define the load job configuration
job_config = bigquery.QueryJobConfig(
    destination=table_ref,
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
    schema_update_options=[
        bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION
    ]
)

# Define the SQL query
insert_query = f"SELECT * FROM `{dataset_name}.{table_name}`"

# Run the load job and wait for the job to complete
load_job = client.query(insert_query, job_config=job_config)

# Wait for the job to complete
job.result()
total_rows = job.output_rows
print(f"Big Query Rows {total_rows} processed into {load_dataset_name}.{load_table_name}")

Total rows deleted from extract_layer.all_api_sportdevs_fb_matches: 6634
Big Query Rows 6634 processed into extract_layer.all_api_sportdevs_fb_matches


In [11]:
# Create and write to line ups file extract data into BIQ QUERY extract_layer lineups table

# Set the target dataset and table names
dataset_name = 'extract_layer'
table_name = 'api_sportdevs_fb_lineups'
table_ref = client.dataset(dataset_name).table(table_name)

def send_requests_in_batches(ids, api_endpoint, batch_size=50):
    total_ids = len(ids)
    total_batches = math.ceil(total_ids / batch_size)
    print(f"Total ids: {total_ids}")
    print(f"Total batches: {total_batches}")

    batches = [ids[i:i+batch_size] for i in range(0, len(ids), batch_size)]
    batch_count = 1

    with open(f"{league_name}_lineups_data.json", 'w') as lineups_file:
        for batch in batches:
            batch_ids = ','.join(map(str, batch))
            url = f"{api_endpoint}?id=in.({batch_ids})"
            response = requests.get(url, headers=headers)
            lineups_data = response.json()

            # Prepare the data to be inserted
            for lineup in lineups_data:
                lineups_file.write(json.dumps(lineup) + '\n')

            print(f"API Batch {batch_count}/{total_batches} processed.")

            batch_count += 1

# Call batch requests function
ids = lineup_ids
api_endpoint = "https://football.sportdevs.com/matches-lineups"
send_requests_in_batches(ids, api_endpoint)

# Load the final extract data file into the BIGQUERY table
job_config = bigquery.LoadJobConfig(
  source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
  write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
  autodetect=True
)

with open(f"{league_name}_lineups_data.json", 'rb') as lineups_file:
              job = client.load_table_from_file(
                    lineups_file, table_ref, job_config=job_config
                )
# Wait for the job to complete and print message
job.result()
total_rows = job.output_rows
print(f"Big Query Rows {total_rows} processed into {dataset_name}.{table_name}.")


Total ids: 6068
Total batches: 122
API Batch 1/122 processed.
API Batch 2/122 processed.
API Batch 3/122 processed.
API Batch 4/122 processed.
API Batch 5/122 processed.
API Batch 6/122 processed.
API Batch 7/122 processed.
API Batch 8/122 processed.
API Batch 9/122 processed.
API Batch 10/122 processed.
API Batch 11/122 processed.
API Batch 12/122 processed.
API Batch 13/122 processed.
API Batch 14/122 processed.
API Batch 15/122 processed.
API Batch 16/122 processed.
API Batch 17/122 processed.
API Batch 18/122 processed.
API Batch 19/122 processed.
API Batch 20/122 processed.
API Batch 21/122 processed.
API Batch 22/122 processed.
API Batch 23/122 processed.
API Batch 24/122 processed.
API Batch 25/122 processed.
API Batch 26/122 processed.
API Batch 27/122 processed.
API Batch 28/122 processed.
API Batch 29/122 processed.
API Batch 30/122 processed.
API Batch 31/122 processed.
API Batch 32/122 processed.
API Batch 33/122 processed.
API Batch 34/122 processed.
API Batch 35/122 proce

In [12]:
# LOAD line ups extract data into BIQ QUERY load_layer table

# Set the target load dataset and table names
load_dataset_name = 'extract_layer'
load_table_name = 'all_api_sportdevs_fb_lineups'
table_ref = client.dataset(load_dataset_name).table(load_table_name)


# Delete rows already inserted
delete_query = f"DELETE FROM `{load_dataset_name}.{load_table_name}` WHERE id IN (SELECT DISTINCT id FROM `{dataset_name}.{table_name}`)"
delete_job = client.query(delete_query)
delete_result = delete_job.result()
total_rows_deleted = delete_result.num_dml_affected_rows

if total_rows_deleted == 0:
    print(f"No rows were deleted from {load_dataset_name}.{load_table_name}.")
else:
   print(f"Total rows deleted from {load_dataset_name}.{load_table_name}: {total_rows_deleted}")

# Define the load job configuration
job_config = bigquery.QueryJobConfig(
    destination=table_ref,
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
    schema_update_options=[
        bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION
    ]
)

# Define the SQL query
insert_query = f"SELECT * FROM `{dataset_name}.{table_name}`"

# Run the load job and wait for the job to complete
load_job = client.query(insert_query, job_config=job_config)

# Wait for the job to complete
job.result()
total_rows = job.output_rows

print(f"Big Query Rows {total_rows} processed into {load_dataset_name}.{load_table_name}")






Total rows deleted from extract_layer.all_api_sportdevs_fb_lineups: 6039
Big Query Rows 6068 processed into extract_layer.all_api_sportdevs_fb_lineups
